<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Zadanie" data-toc-modified-id="Zadanie-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Zadanie</a></span></li><li><span><a href="#Biblioteki-+-GPU" data-toc-modified-id="Biblioteki-+-GPU-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Biblioteki + GPU</a></span></li><li><span><a href="#Rozwiązanie-1---3-warstwowa-sieć-neuronowa" data-toc-modified-id="Rozwiązanie-1---3-warstwowa-sieć-neuronowa-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Rozwiązanie 1 - 3-warstwowa sieć neuronowa</a></span></li><li><span><a href="#Rozwiązanie-2---regresja-liniowa" data-toc-modified-id="Rozwiązanie-2---regresja-liniowa-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Rozwiązanie 2 - regresja liniowa</a></span></li></ul></div>

# Zadanie
+ Przewidzenie jakości wina na podstawie jego parametrów.
+ Zbiór danych: 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'

In [1]:
import pandas as pd

df = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv',
    delimiter=";")

df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


# Biblioteki + GPU

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

In [3]:
torch.cuda.is_available()

True

In [4]:
device = torch.device("cuda")
device

device(type='cuda')

+ Dodanie seedu:

In [5]:
# GPU operations have a separate seed we also want to set
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

# Additionally, some operations on a GPU are implemented stochastic for efficiency
# We want to ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.determinstic = True
torch.backends.cudnn.benchmark = False

# Rozwiązanie 1 - 3-warstwowa sieć neuronowa

+ Wstępne przetwarzanie danych:

In [6]:
import torch.utils.data as data

train = df.sample(frac=0.8, random_state=6)  #random state is a seed value
test = df.drop(train.index)

train_dataset = data.TensorDataset(
    torch.from_numpy(train.values[:, :-1]).float(),
    torch.from_numpy(train.values[:, -1]).float())
train_data_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True)

+ Klasa modelu:

In [7]:
class WineClassifier(nn.Module):
    def __init__(self, num_inputs, num_hidden_1, num_hidden_2, num_outputs):
        super().__init__()
        # Initialize the modules we need to build the network
        self.linear1 = nn.Linear(num_inputs, num_hidden_1)
        self.act_fn_1 = nn.Tanh()
        self.linear2 = nn.Linear(num_hidden_1, num_hidden_2)
        self.act_fn_2 = nn.Tanh()
        self.linear3 = nn.Linear(num_hidden_2, num_outputs)

    def forward(self, x):
        # Perform the calculation of the model to determine the prediction
        x = self.linear1(x)
        x = self.act_fn_1(x)
        x = self.linear2(x)
        x = self.act_fn_2(x)
        x = self.linear3(x)
        return x

+ Model z optymalizacją i stratą:

In [8]:
model_wine = WineClassifier(num_inputs=train_dataset[0][0].shape[0],
                            num_hidden_1=400,
                            num_hidden_2=200,
                            num_outputs=1)
# model_wine = SimpleClassifier(num_inputs=train_dataset[0][0].shape[0], num_hidden = 16, num_outputs=1)
model_wine.to(device)
optimizer_wine = torch.optim.SGD(model_wine.parameters(), lr=0.0006)
loss_f = nn.L1Loss()

+ Trening:

In [9]:
model_wine.train()

# Training loop
for epoch in range(250):
    for data_inputs, data_labels in train_data_loader:
        data_labels = data_labels.unsqueeze(dim=0)
        data_inputs = data_inputs.to(device)
        data_labels = data_labels.to(device)

        preds = model_wine(data_inputs)

        loss = loss_f(preds, data_labels.T)

        optimizer_wine.zero_grad()

        loss.backward()

        optimizer_wine.step()
    if epoch % 30 == 0:
        print(f"Epoch: {epoch}, loss: {loss.item():.3}")

Epoch: 0, loss: 0.721
Epoch: 30, loss: 0.634
Epoch: 60, loss: 0.657
Epoch: 90, loss: 0.486
Epoch: 120, loss: 0.495
Epoch: 150, loss: 0.579
Epoch: 180, loss: 0.814
Epoch: 210, loss: 0.51
Epoch: 240, loss: 0.713


+ Test modelu:

In [10]:
test_dataset = data.TensorDataset(
    torch.from_numpy(test.values[:, :-1]).float(),
    torch.from_numpy(test.values[:, -1]).float())

test_data_loader = data.DataLoader(test_dataset,
                                   batch_size=1,
                                   shuffle=False,
                                   drop_last=False)
model_wine.eval()  # Set model to eval mode
true_preds, num_preds = 0., 0.

with torch.no_grad():  # Deactivate gradients for the following code
    for data_inputs, data_labels in test_data_loader:

        # Determine prediction of model on test set inputs
        data_inputs, data_labels = data_inputs.to(device), data_labels.to(
            device)
        data_labels = data_labels.unsqueeze(dim=0)
        preds = model_wine(data_inputs)
        preds = preds.squeeze(dim=1)
        # rounding because only possible values are [3,4,5,6,7,8,9]
        preds = preds.round()

        # Keeping records of predictions
        true_preds += (preds == data_labels).sum()
        num_preds += data_labels.shape[0]

acc = true_preds / num_preds
print(f"Accuracy of the model: {100.0*acc:4.2f}%")

Accuracy of the model: 45.31%


+ Niestety rozwiązanie niepoprawne - funkcja wypluwa w większości 6 niezależnie od x:

In [11]:
mult_matr = test_dataset[:][0].cpu()
mult_matr.cpu()
model_wine.cpu()
with torch.no_grad():
    Y = model_wine(mult_matr)
Y.T[0][:25].round()

tensor([6., 6., 6., 6., 5., 5., 6., 6., 6., 6., 5., 6., 6., 6., 6., 6., 6., 6.,
        6., 6., 6., 6., 6., 6., 6.])

# Rozwiązanie 2 - regresja liniowa

+ Dane:

In [12]:
train = df.sample(frac=0.8, random_state=6)  #random state is a seed value
test = df.drop(train.index)
train_dataset = data.TensorDataset(
    torch.from_numpy(train.values[:, :-1]).float(),
    torch.from_numpy(train.values[:, -1]).float())
train_data_loader = data.DataLoader(train_dataset, batch_size=16, shuffle=True)

+ Klasa modelu:

In [13]:
class WineClassifier2(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        # Initialize the modules we need to build the network
        self.linear = nn.Linear(num_inputs, num_outputs)

    def forward(self, x):
        # Perform the calculation of the model to determine the prediction
        x = self.linear(x)
        return x

+ Model + optymalizator + funkcja straty:

In [14]:
model_wine2 = WineClassifier2(num_inputs=train_dataset[0][0].shape[0],
                              num_outputs=1)
model_wine2.to(device)
optimizer_wine2 = torch.optim.Adam(model_wine2.parameters(), lr=1e-5)
loss_f2 = nn.L1Loss()

+ Uczenie sieci:

In [15]:
model_wine2.train()

# Training loop
for epoch in range(500):
    for data_inputs, data_labels in train_data_loader:
        data_labels = data_labels.unsqueeze(dim=0)
        data_inputs = data_inputs.to(device)
        data_labels = data_labels.to(device)
        preds2 = model_wine2(data_inputs)
        loss2 = loss_f2(preds2, data_labels.T)
        optimizer_wine2.zero_grad()
        loss2.backward()
        optimizer_wine2.step()
    if epoch % 50 == 0:
        print(f"Epoch: {epoch}, loss: {loss2.item():.3}")

Epoch: 0, loss: 2.5
Epoch: 50, loss: 1.86
Epoch: 100, loss: 0.937
Epoch: 150, loss: 0.573
Epoch: 200, loss: 0.42
Epoch: 250, loss: 0.653
Epoch: 300, loss: 0.661
Epoch: 350, loss: 0.686
Epoch: 400, loss: 0.712
Epoch: 450, loss: 0.489


+ Ewaluacja wyników:

In [16]:
test_dataset = data.TensorDataset(
    torch.from_numpy(test.values[:, :-1]).float(),
    torch.from_numpy(test.values[:, -1]).float())
test_data_loader = data.DataLoader(test_dataset,
                                   batch_size=1,
                                   shuffle=False,
                                   drop_last=False)
model_wine2.eval()  # Set model to eval mode
true_preds, num_preds = 0., 0.

with torch.no_grad():  # Deactivate gradients for the following code
    for data_inputs, data_labels in test_data_loader:

        # Determine prediction of model on test set inputs
        data_inputs, data_labels = data_inputs.to(device), data_labels.to(
            device)
        data_labels = data_labels.unsqueeze(dim=0)
        preds = model_wine2(data_inputs)
        preds = preds.squeeze(dim=1)
        # rounding because only possible values are [3,4,5,6,7,8,9]
        preds = preds.round()

        # Keeping records of predictions
        true_preds += (preds == data_labels).sum()
        num_preds += data_labels.shape[0]

acc = true_preds / num_preds
print(f"Accuracy of the model: {100.0*acc:4.2f}%")

Accuracy of the model: 48.98%


+ Pierwsze 30 sklasyfikowanych ocen:

In [17]:
mult_matr2 = test_dataset[:][0].cpu()
mult_matr2.cpu()
model_wine2.cpu()
with torch.no_grad():
    Y2 = model_wine2(mult_matr)
Y2.T[0][:55].round()

tensor([6., 7., 6., 6., 5., 6., 6., 6., 5., 5., 5., 6., 5., 6., 5., 6., 6., 6.,
        5., 6., 5., 6., 6., 6., 6., 6., 6., 6., 6., 5., 6., 6., 6., 5., 5., 5.,
        6., 6., 5., 5., 5., 6., 6., 6., 5., 6., 6., 6., 6., 5., 5., 6., 6., 6.,
        5.])